In [1]:
from glob import glob
from tqdm.notebook import tqdm
import math

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
from fastai.layers import SigmoidRange

## Evaluation

In [2]:
def rmspe_metric(y_true, y_pred):

    """
    Calculate root mean squared percentage error between ground-truth and predictions

    Parameters
    ----------
    y_true [array-like of shape (n_samples)]: Ground-truth
    y_pred [array-like of shape (n_samples)]: Predictions

    Returns
    -------
    rmspe (float): Root mean squared percentage error
    """

    rmspe = np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))
    return rmspe


def evaluate_predictions(predictions_column, evaluate_stock=False):
    
    for fold in sorted(df_train['fold'].unique()):

        _, val_idx = df_train.loc[df_train['fold'] != fold].index, df_train.loc[df_train['fold'] == fold].index
        fold_score = rmspe_metric(df_train.loc[val_idx, 'target'], df_train.loc[val_idx, predictions_column])
        print(f'Fold {fold} - RMSPE: {fold_score:.6}')

    oof_score = rmspe_metric(df_train['target'], df_train[predictions_column])
    print(f'{"-" * 30}\nOOF RMSPE: {oof_score:.6}\n{"-" * 30}')

    if evaluate_stock:
        for stock_id in df_train['stock_id'].unique():
            df_stock = df_train.loc[df_train['stock_id'] == stock_id, :]
            stock_oof_score = rmspe_metric(df_stock['target'], df_stock[predictions_column])
            print(f'Stock {stock_id} - OOF RMSPE: {stock_oof_score:.6}')


## Preprocessing

In [3]:
class PreprocessingPipeline:
    
    def __init__(self, df_train, df_test):
        
        self.df_train = df_train.copy(deep=True)
        self.df_test = df_test.copy(deep=True)
        
    def _label_encode(self):

        # Encoding stock_id for embeddings
        le = LabelEncoder()
        self.df_train['stock_id_encoded'] = le.fit_transform(self.df_train['stock_id'].values)
        self.df_test['stock_id_encoded'] = le.transform(self.df_test['stock_id'].values)
    
    def _get_folds(self):
        
        # Load pre-computed folds
        self.df_train['fold'] = pd.read_csv('../input/optiver-realized-volatility-dataset/folds.csv')['fold_group']
        self.df_train['fold'] = self.df_train['fold'].astype(np.uint8)
        
    def transform(self):
        
        self._label_encode()
        self._get_folds()
        
        return self.df_train, self.df_test


In [4]:
train_test_dtypes = {
    'stock_id': np.uint8,
    'time_id': np.uint16,
    'target': np.float64
}

df_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv', dtype=train_test_dtypes)
df_test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv', usecols=['stock_id', 'time_id'], dtype=train_test_dtypes)

# Using only first row of test set if it is the placeholder
# Other rows break the pipeline since some of the time buckets don't exist in book data
if df_test.shape[0] == 3:
    df_test = df_test.head(1)

preprocessing_pipeline = PreprocessingPipeline(df_train, df_test)
df_train, df_test = preprocessing_pipeline.transform()

print(f'Training Set Shape: {df_train.shape} - Memory Usage: {df_train.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'Test Set Shape: {df_test.shape} - Memory Usage: {df_test.memory_usage().sum() / 1024 ** 2:.2f} MB')

Training Set Shape: (428932, 5) - Memory Usage: 8.18 MB
Test Set Shape: (1, 3) - Memory Usage: 0.00 MB


## 1D CNN

In [5]:
class Conv1dBlock(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride, skip_connection=False):

        super(Conv1dBlock, self).__init__()

        self.skip_connection = skip_connection
        self.conv_block = nn.Sequential(
            nn.Conv1d(
                in_channels,
                out_channels,
                kernel_size=(kernel_size,),
                stride=(stride,),
                padding=(kernel_size // 2,),
                padding_mode='replicate',
                bias=True
            ),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
        )
        self.downsample = nn.Sequential(
            nn.Conv1d(
                in_channels,
                out_channels,
                kernel_size=(1,),
                stride=(stride,),
                bias=False
            ),
            nn.BatchNorm1d(out_channels)
        )
        self.relu = nn.ReLU()

    def forward(self, x):

        output = self.conv_block(x)
        if self.skip_connection:
            x = self.downsample(x)
            output += x
        output = self.relu(output)

        return output


class Conv1dLayers(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, depth_scale, width_scale, skip_connection, initial=False):

        super(Conv1dLayers, self).__init__()

        depth = int(math.ceil(2 * depth_scale))
        width = int(math.ceil(out_channels * width_scale))

        if initial:
            layers = [
                Conv1dBlock(
                    in_channels=in_channels,
                    out_channels=width,
                    kernel_size=kernel_size,
                    stride=2,
                    skip_connection=skip_connection
                )
            ]
        else:
            layers = [
                Conv1dBlock(
                    in_channels=(int(math.ceil(in_channels * width_scale))),
                    out_channels=width,
                    kernel_size=kernel_size,
                    stride=2,
                    skip_connection=skip_connection
                )
            ]

        for _ in range(depth - 1):
            layers += [
                Conv1dBlock(
                    in_channels=width,
                    out_channels=width,
                    kernel_size=kernel_size,
                    stride=1,
                    skip_connection=skip_connection
                )
            ]

        self.conv_layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.conv_layers(x)
    

class NonLocalBlock1d(nn.Module):

    def __init__(self, in_channels, inter_channels=None, mode='embedded'):

        super(NonLocalBlock1d, self).__init__()

        self.in_channels = in_channels
        self.inter_channels = inter_channels
        self.mode = mode

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
            if self.inter_channels == 0:
                self.inter_channels = 1

        self.g = nn.Conv1d(
            in_channels=self.in_channels,
            out_channels=self.inter_channels,
            kernel_size=(1,),
            stride=(1,),
            padding=0
        )
        self.W_z = nn.Sequential(
            nn.Conv1d(
                in_channels=self.inter_channels,
                out_channels=self.in_channels,
                kernel_size=(1,),
                stride=(1,),
                padding=0
            ),
            nn.BatchNorm1d(self.in_channels)
        )
        nn.init.constant_(self.W_z[1].weight, 0)
        nn.init.constant_(self.W_z[1].bias, 0)

        if self.mode == 'embedded' or self.mode == 'dot' or self.mode == 'concatenate':

            self.theta = nn.Conv1d(
                in_channels=self.in_channels,
                out_channels=self.inter_channels,
                kernel_size=(1,),
                stride=(1,),
                padding=0
            )
            self.phi = nn.Conv1d(
                in_channels=self.in_channels,
                out_channels=self.inter_channels,
                kernel_size=(1,),
                stride=(1,),
                padding=0
            )

        if self.mode == 'concatenate':

            self.W_f = nn.Sequential(
                nn.Conv2d(
                    in_channels=self.inter_channels * 2,
                    out_channels=1,
                    kernel_size=(1, 1),
                    stride=(1, 1),
                    padding=0,
                    bias=False
                ),
                nn.ReLU()
            )

    def forward(self, x):

        batch_size = x.size(0)

        g_x = self.g(x).view(batch_size, self.inter_channels, -1)
        g_x = g_x.permute(0, 2, 1)

        if self.mode == 'gaussian':

            theta_x = x.view(batch_size, self.in_channels, -1)
            phi_x = x.view(batch_size, self.in_channels, -1)
            theta_x = theta_x.permute(0, 2, 1)
            f = torch.matmul(theta_x, phi_x)

        elif self.mode == 'embedded' or self.mode == 'dot':

            theta_x = self.theta(x).view(batch_size, self.inter_channels, -1)
            phi_x = self.phi(x).view(batch_size, self.inter_channels, -1)
            theta_x = theta_x.permute(0, 2, 1)
            f = torch.matmul(theta_x, phi_x)

        elif self.mode == 'concatenate':

            theta_x = self.theta(x).view(batch_size, self.inter_channels, -1, 1)
            phi_x = self.phi(x).view(batch_size, self.inter_channels, 1, -1)
            h = theta_x.size(2)
            w = phi_x.size(3)
            theta_x = theta_x.repeat(1, 1, 1, w)
            phi_x = phi_x.repeat(1, 1, h, 1)
            concat = torch.cat([theta_x, phi_x], dim=1)
            f = self.W_f(concat)
            f = f.view(f.size(0), f.size(2), f.size(3))

        if self.mode == 'gaussian' or self.mode == 'embedded':
            f_div_C = F.softmax(f, dim=-1)
        elif self.mode == 'dot' or self.mode == 'concatenate':
            N = f.size(-1)
            f_div_C = f / N

        y = torch.matmul(f_div_C, g_x)
        y = y.permute(0, 2, 1).contiguous()
        y = y.view(batch_size, self.inter_channels, *x.size()[2:])

        W_y = self.W_z(y)
        z = W_y + x

        return z


class CNN1DModel(nn.Module):

    def __init__(self, in_channels, out_channels, use_stock_id, stock_embedding_dims, alpha, beta, phi):

        super(CNN1DModel, self).__init__()

        # Stock embeddings
        self.use_stock_id = use_stock_id
        self.stock_embedding_dims = stock_embedding_dims
        self.stock_embeddings = nn.Embedding(num_embeddings=113, embedding_dim=self.stock_embedding_dims)
        self.dropout = nn.Dropout(0.25)

        # Model scaling
        depth_scale = alpha ** phi
        width_scale = beta ** phi
        self.out_channels = int(math.ceil(out_channels * width_scale))

        # Convolutional layers
        self.conv_layers1 = Conv1dLayers(
            in_channels=in_channels,
            out_channels=32,
            kernel_size=5,
            depth_scale=depth_scale,
            width_scale=width_scale,
            skip_connection=False,
            initial=True
        )
        self.conv_layers2 = Conv1dLayers(
            in_channels=32,
            out_channels=64,
            kernel_size=7,
            depth_scale=depth_scale,
            width_scale=width_scale,
            skip_connection=False,
            initial=False
        )
        self.conv_layers3 = Conv1dLayers(
            in_channels=64,
            out_channels=128,
            kernel_size=9,
            depth_scale=depth_scale,
            width_scale=width_scale,
            skip_connection=False,
            initial=False
        )
        self.conv_layers4 = Conv1dLayers(
            in_channels=128,
            out_channels=self.out_channels,
            kernel_size=11,
            depth_scale=depth_scale,
            width_scale=width_scale,
            skip_connection=False,
            initial=False
        )
        
        # Non-local blocks
        self.nl_block1 = NonLocalBlock1d(in_channels=32, mode='embedded')
        self.nl_block2 = NonLocalBlock1d(in_channels=64, mode='embedded')
        self.nl_block3 = NonLocalBlock1d(in_channels=128, mode='embedded')
        self.nl_block4 = NonLocalBlock1d(in_channels=self.out_channels, mode='embedded')
        
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Sequential(
            nn.Linear(256 + self.stock_embedding_dims, 1, bias=True),
            SigmoidRange(0, 0.1)
        )

    def forward(self, stock_ids, sequences):

        x = torch.transpose(sequences, 1, 2)
        x = self.conv_layers1(x)
        x = self.nl_block1(x)
        x = self.conv_layers2(x)
        x = self.nl_block2(x)
        x = self.conv_layers3(x)
        x = self.nl_block3(x)
        x = self.conv_layers4(x)
        x = self.nl_block4(x)
        x = self.pooling(x)
        x = x.view(-1, x.shape[1])

        if self.use_stock_id:
            embedded_stock_ids = self.stock_embeddings(stock_ids)
            x = torch.cat([x, self.dropout(embedded_stock_ids)], dim=1)

        output = self.head(x)
        return output.view(-1)


## RNN

In [6]:
class SelfAttention(nn.Module):

    def __init__(self, attention_size):

        super(SelfAttention, self).__init__()

        self.attention_weights = nn.Parameter(torch.FloatTensor(attention_size))
        nn.init.uniform_(self.attention_weights.data, -0.005, 0.005)

        self.softmax = nn.Softmax(dim=-1)
        self.relu = nn.ReLU()

    def forward(self, x):

        attentions = self.relu(x.matmul(self.attention_weights))
        attentions = self.softmax(attentions)
        weighted = torch.mul(x, attentions.unsqueeze(-1).expand_as(x))
        representations = weighted.sum(1).squeeze()

        return representations, attentions

    
class RNNModel(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, use_stock_id, stock_embedding_dims):

        super(RNNModel, self).__init__()

        # Stock embeddings
        self.use_stock_id = use_stock_id
        self.stock_embedding_dims = stock_embedding_dims
        self.stock_embeddings = nn.Embedding(num_embeddings=113, embedding_dim=self.stock_embedding_dims)
        self.dropout = nn.Dropout(0.25)

        # Recurrent neural network
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.gru = nn.GRU(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            dropout=0,
            bidirectional=False,
            batch_first=True
        )

        # Sequence self attention
        self.attention = SelfAttention(attention_size=self.hidden_size)
        
        self.head = nn.Sequential(
            nn.Linear(self.hidden_size + self.stock_embedding_dims, 1, bias=True),
            SigmoidRange(0, 0.1)
        )

    def forward(self, stock_ids, sequences):
        
        h_n0 = torch.zeros(self.num_layers, sequences.size(0), self.hidden_size).to(self.device)
        gru_output, h_n = self.gru(sequences, h_n0)
        representations, attentions = self.attention(gru_output)

        if self.use_stock_id:
            embedded_stock_ids = self.stock_embeddings(stock_ids)
            x = torch.cat([representations.view(1, -1), self.dropout(embedded_stock_ids)], dim=1)

        output = self.head(x)
        return output.view(-1)


## Models

In [7]:
# Normalizing sequences with global means and stds
book_means = np.array([
    # Raw sequences
    0.99969482421875, 1.000321388244629, 0.9995064735412598, 1.0005191564559937,
    769.990177708821, 766.7345672818379, 959.3416027831918, 928.2202512713748,
    # Absolute log returns of raw sequences
    5.05890857311897e-05, 5.1026330766035244e-05, 5.74059049540665e-05, 5.8218309277435765e-05,
    0.3967152245253066, 0.39100519899866804, 0.3239659116907835, 0.31638538484106116,
    # Weighted average prices
    1.0000068043192514, 1.0000055320253616, 1.000006872969592,
    # Absolute log returns of weighted average prices
    8.211420490291096e-05, 0.00011112522790786203, 8.236187150264073e-05
])
book_stds = np.array([
    # Raw sequences
    0.0036880988627672195, 0.003687119111418724, 0.0037009266670793295, 0.0036990800872445107,
    5354.051690318169, 4954.947103063445, 6683.816183660414, 5735.299917793827,
    # Absolute log returns of raw sequences
    0.00016576898633502424, 0.00016801751917228103, 0.0001837657910073176, 0.0001868011022452265,
    0.9121719707304721, 0.8988021131995019, 0.8415323589617927, 0.8244750862945265,
    # Weighted average prices
    0.003689893218043926, 0.00370745215558702, 0.0036913980961173682,
    # Absolute log returns of weighted average prices
    0.00021108155612872302, 0.00029320157822289604, 0.00019975085953727163
])
trade_means = np.array([0.999971866607666, 352.9736760331942, 4.1732040971227145])
trade_stds = np.array([0.004607073962688446, 1041.9441951057488, 7.79955795393431])

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# CNN 1D
cnn_parameters = {
    'in_channels': 25,
    'out_channels': 256,
    'use_stock_id': True,
    'stock_embedding_dims': 16,
    'alpha': 1,
    'beta': 1,
    'phi': 1
}

print(f'CNN1D Models\n{"-" * 12}')
cnn_models = []
for model_path in sorted(glob(f'../input/optiver-realized-volatility-dataset/cnn1d/*.pt')):
    print(f'Loading model {model_path} into memory')
    model = CNN1DModel(**cnn_parameters)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    cnn_models.append(model)
    
# RNN
rnn_parameters = {
    'input_size': 25,
    'hidden_size': 64,
    'num_layers': 3,
    'use_stock_id': True,
    'stock_embedding_dims': 16,
}

print(f'\nRNN Models\n{"-" * 10}')
rnn_models = []
for model_path in sorted(glob(f'../input/optiver-realized-volatility-dataset/rnn/*.pt')):
    print(f'Loading model {model_path} into memory')
    model = RNNModel(**rnn_parameters)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    rnn_models.append(model)


CNN1D Models
------------
Loading model ../input/optiver-realized-volatility-dataset/cnn1d/cnn1d_fold1.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/cnn1d/cnn1d_fold2.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/cnn1d/cnn1d_fold3.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/cnn1d/cnn1d_fold4.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/cnn1d/cnn1d_fold5.pt into memory

RNN Models
----------
Loading model ../input/optiver-realized-volatility-dataset/rnn/rnn_fold1.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/rnn/rnn_fold2.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/rnn/rnn_fold3.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/rnn/rnn_fold4.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/rnn/rnn_fold5.pt into memory


In [8]:
# Loading pre-computed train predictions and evaluate them
df_train['cnn1d_predictions'] = pd.read_csv('../input/optiver-realized-volatility-dataset/cnn1d/cnn1d_predictions.csv').values
df_train['rnn_predictions'] = pd.read_csv('../input/optiver-realized-volatility-dataset/rnn/rnn_predictions.csv').values

print(f'CNN1D\n{"-" * 5}')
evaluate_predictions('cnn1d_predictions')

print(f'\nRNN\n{"-" * 3}')
evaluate_predictions('rnn_predictions')

print(f'\nBlend\n{"-" * 5}')
df_train['blend_predictions'] = (df_train['cnn1d_predictions'] * 0.5) + (df_train['rnn_predictions'] * 0.5)
evaluate_predictions('blend_predictions')

CNN1D
-----
Fold 1 - RMSPE: 0.234049
Fold 2 - RMSPE: 0.238894
Fold 3 - RMSPE: 0.225608
Fold 4 - RMSPE: 0.223012
Fold 5 - RMSPE: 0.222544
------------------------------
OOF RMSPE: 0.228914
------------------------------

RNN
---
Fold 1 - RMSPE: 0.231432
Fold 2 - RMSPE: 0.242344
Fold 3 - RMSPE: 0.225047
Fold 4 - RMSPE: 0.225377
Fold 5 - RMSPE: 0.220146
------------------------------
OOF RMSPE: 0.228997
------------------------------

Blend
-----
Fold 1 - RMSPE: 0.230628
Fold 2 - RMSPE: 0.238636
Fold 3 - RMSPE: 0.223709
Fold 4 - RMSPE: 0.221811
Fold 5 - RMSPE: 0.219708
------------------------------
OOF RMSPE: 0.227004
------------------------------


In [9]:
book_features = ['bid_price1', 'ask_price1', 'bid_price2', 'ask_price2','bid_size1', 'ask_size1', 'bid_size2', 'ask_size2']
trade_features = ['price', 'size', 'order_count']
stock_id_mapping = df_train.set_index('stock_id')['stock_id_encoded'].to_dict()

for stock_id in tqdm(df_test['stock_id'].unique()):
    
    df_stock = df_test.loc[df_test['stock_id'] == stock_id]
    df_book = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id={stock_id}')
    df_trade = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id={stock_id}')
    stock_time_buckets = df_test.loc[df_test['stock_id'] == stock_id, 'time_id'].reset_index(drop=True)
    missing_time_buckets = stock_time_buckets[~stock_time_buckets.isin(df_trade['time_id'])]
    df_trade = df_trade.merge(missing_time_buckets, how='outer')
    
    # Iterating over time_ids
    for time_id in df_stock['time_id'].unique():
        
        # Resample order book to 600 seconds, forward fill and back fill for edge cases
        df_book_time_bucket = df_book.loc[df_book['time_id'] == time_id]
        df_book_time_bucket = df_book_time_bucket.set_index(['seconds_in_bucket'])
        df_book_time_bucket = df_book_time_bucket.reindex(np.arange(0, 600), method='ffill').fillna(method='bfill')
        
        # Sequences from book data
        book_sequences = df_book_time_bucket.reset_index(drop=True)[book_features].values
        
        # Absolute log returns of raw sequences
        book_bid_price1_log = np.log(book_sequences[:, 0])
        book_bid_price1_absolute_log_returns = np.abs(np.diff(book_bid_price1_log, prepend=[book_bid_price1_log[0]]))
        book_ask_price1_log = np.log(book_sequences[:, 1])
        book_ask_price1_absolute_log_returns = np.abs(np.diff(book_ask_price1_log, prepend=[book_ask_price1_log[0]]))
        book_bid_price2_log = np.log(book_sequences[:, 2])
        book_bid_price2_absolute_log_returns = np.abs(np.diff(book_bid_price2_log, prepend=[book_bid_price2_log[0]]))
        book_ask_price2_log = np.log(book_sequences[:, 3])
        book_ask_price2_absolute_log_returns = np.abs(np.diff(book_ask_price2_log, prepend=[book_ask_price2_log[0]]))
        book_bid_size1_log = np.log(book_sequences[:, 4])
        book_bid_size1_absolute_log_returns = np.abs(np.diff(book_bid_size1_log, prepend=[book_bid_size1_log[0]]))
        book_ask_size1_log = np.log(book_sequences[:, 5])
        book_ask_size1_absolute_log_returns = np.abs(np.diff(book_ask_size1_log, prepend=[book_ask_size1_log[0]]))
        book_bid_size2_log = np.log(book_sequences[:, 6])
        book_bid_size2_absolute_log_returns = np.abs(np.diff(book_bid_size2_log, prepend=[book_bid_size2_log[0]]))
        book_ask_size2_log = np.log(book_sequences[:, 7])
        book_ask_size2_absolute_log_returns = np.abs(np.diff(book_ask_size2_log, prepend=[book_ask_size2_log[0]]))

        # Weighted average prices
        book_wap1 = (book_sequences[:, 0] * book_sequences[:, 5] + book_sequences[:, 1] * book_sequences[:, 4]) /\
                    (book_sequences[:, 4] + book_sequences[:, 5])
        book_wap2 = (book_sequences[:, 2] * book_sequences[:, 7] + book_sequences[:, 3] * book_sequences[:, 6]) /\
                    (book_sequences[:, 6] + book_sequences[:, 7])
        book_wap3 = ((book_sequences[:, 0] * book_sequences[:, 5] + book_sequences[:, 1] * book_sequences[:, 4]) +
                     (book_sequences[:, 2] * book_sequences[:, 7] + book_sequences[:, 3] * book_sequences[:, 6])) /\
                    (book_sequences[:, 4] + book_sequences[:, 5] + book_sequences[:, 6] + book_sequences[:, 7])

        # Absolute log returns of weighted average prices
        book_wap1_log = np.log(book_wap1)
        book_wap1_absolute_log_returns = np.abs(np.diff(book_wap1_log, prepend=[book_wap1_log[0]]))
        book_wap2_log = np.log(book_wap2)
        book_wap2_absolute_log_returns = np.abs(np.diff(book_wap2_log, prepend=[book_wap2_log[0]]))
        book_wap3_log = np.log(book_wap3)
        book_wap3_absolute_log_returns = np.abs(np.diff(book_wap3_log, prepend=[book_wap3_log[0]]))

        book_sequences = np.hstack([
            book_sequences,
            book_bid_price1_absolute_log_returns.reshape(-1, 1),
            book_ask_price1_absolute_log_returns.reshape(-1, 1),
            book_bid_price2_absolute_log_returns.reshape(-1, 1),
            book_ask_price2_absolute_log_returns.reshape(-1, 1),
            book_bid_size1_absolute_log_returns.reshape(-1, 1),
            book_ask_size1_absolute_log_returns.reshape(-1, 1),
            book_bid_size2_absolute_log_returns.reshape(-1, 1),
            book_ask_size2_absolute_log_returns.reshape(-1, 1),
            book_wap1.reshape(-1, 1),
            book_wap2.reshape(-1, 1),
            book_wap3.reshape(-1, 1),
            book_wap1_absolute_log_returns.reshape(-1, 1),
            book_wap2_absolute_log_returns.reshape(-1, 1),
            book_wap3_absolute_log_returns.reshape(-1, 1),
        ])
        book_sequences = (book_sequences - book_means) / book_stds
        
        # Resample trade data to 600 seconds and fill missing values with 0
        df_trade_time_bucket = df_trade.loc[df_trade['time_id'] == time_id]
        df_trade_time_bucket = df_trade_time_bucket.set_index(['seconds_in_bucket'])
        df_trade_time_bucket = df_trade_time_bucket.reindex(np.arange(0, 600)).fillna(0)
        
        # Sequences from trade data
        trade_sequences = df_trade_time_bucket.reset_index(drop=True)[trade_features].values
        # Not normalizing zero values in trade data
        trade_sequences[trade_sequences[:, 0] != 0, :] = (trade_sequences[trade_sequences[:, 0] != 0, :] - trade_means) / trade_stds
        
        # Concatenate book and trade sequences
        sequences = np.hstack([book_sequences, trade_sequences])
        sequences = torch.as_tensor(sequences.reshape(1, 600, 25), dtype=torch.float)
        sequences = sequences.to(device)
        
        stock_id_encoded = torch.as_tensor([stock_id_mapping[stock_id]], dtype=torch.long)
        stock_id_encoded = stock_id_encoded.to(device)
        
        cnn_prediction = 0
        for model in cnn_models:
            with torch.no_grad():
                cnn_model_prediction = model(stock_id_encoded, sequences).detach().cpu().numpy()
                cnn_prediction += (cnn_model_prediction[0] / 5)
        df_test.loc[(df_test['stock_id'] == stock_id) & (df_test['time_id'] == time_id), 'cnn1d_predictions'] = cnn_prediction
                
        rnn_prediction = 0
        for model in rnn_models:
            with torch.no_grad():
                rnn_model_prediction = model(stock_id_encoded, sequences).detach().cpu().numpy()
                rnn_prediction += (rnn_model_prediction[0] / 5)
        df_test.loc[(df_test['stock_id'] == stock_id) & (df_test['time_id'] == time_id), 'rnn_predictions'] = rnn_prediction


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
df_test['target'] = (df_test['cnn1d_predictions'] * 0.5) + (df_test['rnn_predictions'] * 0.5)
df_test['row_id'] = df_test['stock_id'].astype(str) + '-' + df_test['time_id'].astype(str)
df_test[['row_id', 'target']].to_csv('submission.csv', index=False)

In [11]:
df_test[['row_id', 'target']]

,row_id,target
0,0-4,0.001076
